<a href="https://colab.research.google.com/github/will-cotton4/DS-Unit-1-Sprint-1-Dealing-With-Data/blob/master/module1-afirstlookatdata/LS_DS_111_A_First_Look_at_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - A First Look at Data



## Lecture - let's explore Python DS libraries and examples!

The Python Data Science ecosystem is huge. You've seen some of the big pieces - pandas, scikit-learn, matplotlib. What parts do you want to see more of?

In [0]:
# TODO - we'll be doing this live, taking requests
# and reproducing what it is to look up and learn things



## Assignment - now it's your turn

Pick at least one Python DS library, and using documentation/examples reproduce in this notebook something cool. It's OK if you don't fully understand it or get it 100% working, but do put in effort and look things up.

In [26]:
### For this assignment, I decided to work using the SymPy library, the documentation
### of which can be found here: https://docs.sympy.org/latest/index.html

from sympy import *
import numpy as np

x, y, z = symbols("x y z")

### The following function approximates the sum of a convergent alternating series within
### the desired error. 

### We know from calculus that a convergent alternating series can be approximated
### by taking the partial sum (the first n terms), which is within the absolute value of the
### next term (i.e. |a_(n+1)|) of the actual sum of the entire series. That is to say, 
### the sum of the series is between the nth partial sum minus the (n+1)th term
### and the nth partial sum plus the (n+1)th term.

### Side-note: because an alternating series only converges if it's monotonically
### decreasing, the looping logic for this problem is really easy: just check
### successive absolute values to see when our error is small enough, adding intermediate
### values all the while, then returning the result once we've gotten to the desired
### range.

### But, of course, this STILL requires more thought than we in our busy lives can account for,
### and for most of us calculus was a long time ago.

### So let's use Python to make our jobs that much easier. 

def approx_alt_series(math_series, error):
  ### Double-check the first value of the series to see if we need to go any further
  ### in our approximation
  test_error = np.absolute(math_series.subs(x,1))
  i=1
  ### Initialize a variable to store our running approximation.
  approx_sum = math_series.subs(x,i)
  
  ### As noted above, run this loop as long as we need to to get our error within
  ### the appropriate bounds.
  
  while(np.absolute(math_series.subs(x,i)) > error):
    
    i+=1
    approx_sum += math_series.subs(x,i)
    test_error = np.absolute(math_series.subs(x,i))
    
  return approx_sum

test_series = ((-1)**x)*x**-2

print("The sum is approximately: {0}".format(float(approx_alt_series(test_series, .01))))

print("The sum is approximately: {0}".format(float(approx_alt_series(test_series, .001))))  
  
print("The sum is approximately: {0}".format(float(approx_alt_series(test_series, .00001))))


The sum is approximately: -0.8179621756109852
The sum is approximately: -0.8219939961054308
The sum is approximately: -0.8224719933971315
The true sum, we all of course remember, is 1.6449340668482264


### Assignment questions

After you've worked on some code, answer the following questions in this text block:

1.  Describe in a paragraph of text what you did and why, as if you were writing an email to somebody interested but nontechnical.

2.  What was the most challenging part of what you did?

3.  What was the most interesting thing you learned?

4.  What area would you like to explore with more time?




## Stretch goals and resources

Following are *optional* things for you to take a look at. Focus on the above assignment first, and make sure to commit and push your changes to GitHub (and since this is the first assignment of the sprint, open a PR as well).

- [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)
- [scikit-learn documentation](http://scikit-learn.org/stable/documentation.html)
- [matplotlib documentation](https://matplotlib.org/contents.html)
- [Awesome Data Science](https://github.com/bulutyazilim/awesome-datascience) - a list of many types of DS resources

Stretch goals:

- Find and read blogs, walkthroughs, and other examples of people working through cool things with data science - and share with your classmates!
- Write a blog post (Medium is a popular place to publish) introducing yourself as somebody learning data science, and talking about what you've learned already and what you're excited to learn more about.